In [1]:
import os
import cv2
import glob
import numpy as np
import matplotlib.pyplot as plt

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
data_path = '/content/drive/MyDrive/Human activity /hra_test/sample_videos'

In [4]:
categoeries = os.listdir(data_path)
labels = [i for i in range(len(categoeries))]
label_dict = dict(zip(categoeries, labels))

In [5]:
print(label_dict)
print(categoeries)
print(labels)

{'cricket': 0, 'Golf': 1, 'yo_yo': 2, 'typing': 3, 'Drumming': 4, 'writing_on_board': 5, 'Handstand': 6, 'Fencing': 7, 'Walkng_witthDog': 8, 'tennis': 9}
['cricket', 'Golf', 'yo_yo', 'typing', 'Drumming', 'writing_on_board', 'Handstand', 'Fencing', 'Walkng_witthDog', 'tennis']
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]


In [6]:
frames = '/content/drive/MyDrive/Human activity /hra_test/frames'

In [7]:
if not os.path.exists(frames):
    os.makedirs(frames)

In [8]:
root_path = '/content/drive/MyDrive/Human activity /hra_test/frames' # Path for writing frames from videos

In [9]:
data=[] # Appending images as array in the list
target=[] #Target list to make prediction

In [12]:
folder_count = 0
img_size=160

In [13]:
for items in categoeries: #looping over folders in categoeries
    path = os.path.join(root_path, items) 
    folder_path = os.path.join(data_path,items)
    img_names = os.listdir(folder_path) # Listing all files inside the directory
    os.mkdir(path)
    img_count= 0
    folder_count +=1
    
    for img_name in img_names:
            cap = cv2.VideoCapture(os.path.join(folder_path, img_name)) # Capcturing video and dividing it by frames
            success, image = cap.read()
            while success:
                success, image = cap.read()
                if not success:
                    break
                
                
                #gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY) # Converting coloured image to greyScale
                #norm = cv2.normalize(gray_image, None, alpha=0,beta=200, norm_type=cv2.NORM_MINMAX) # Normalize image
                #resized_up = cv2.resize(image, down_points, interpolation= cv2.INTER_LINEAR) # Resizing image
                resized=cv2.resize(image,(img_size,img_size))
                cv2.imwrite(path + '/' + items +str(img_count) + '.jpg' , resized) # Writing image into to specified class with its class name 
                data.append(resized)
                target.append(label_dict[items])
                
                img_count += 1

In [14]:
def normalize(x):
    x = np.array(data)
    x = x.astype('float32')
    x = x/255.0
    return x

In [15]:
data_x = normalize(data)

In [16]:
target=np.array(target)

In [20]:
target = target.reshape(-1,)
target[:5]

array([0, 0, 0, 0, 0])

In [21]:
classes = ['cricket', 'Golf', 'yo_yo', 'typing', 'Drumming', 'writing_on_board', 'Handstand', 'Fencing', 'Walkng_witthDog', 'tennis']

In [22]:
data_x.shape

(8450, 160, 160, 3)

In [23]:
target.shape

(8450,)

In [24]:
from sklearn.model_selection import train_test_split
X_train ,X_test ,y_train ,y_test =train_test_split(data_x,target,test_size=0.2)

In [25]:
print("X Train shape :", X_train.shape)
print("X Test shape :", X_test.shape)
print("y Train shape :", y_train.shape)
print("y test shape :", y_test.shape)

X Train shape : (6760, 160, 160, 3)
X Test shape : (1690, 160, 160, 3)
y Train shape : (6760,)
y test shape : (1690,)


In [26]:
import tensorflow as tf
from tensorflow.keras import datasets, layers, models
import matplotlib.pyplot as plt
import numpy as np

In [27]:
ann = models.Sequential([
        layers.Flatten(input_shape=(160,160,3)),
        layers.Dense(3000, activation='relu'),
        layers.Dense(1000, activation='relu'),
        layers.Dense(10, activation='softmax')    
    ])

ann.compile(optimizer='SGD',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [28]:
ann.fit(X_train, y_train, epochs=5)

Epoch 1/5
212/212 [==============================] - 174s 816ms/step - loss: 0.4722 - accuracy: 0.9228
Epoch 2/5
212/212 [==============================] - 172s 810ms/step - loss: 0.0140 - accuracy: 1.0000
Epoch 3/5
212/212 [==============================] - 172s 811ms/step - loss: 0.0068 - accuracy: 1.0000
Epoch 4/5
212/212 [==============================] - 171s 808ms/step - loss: 0.0043 - accuracy: 1.0000
Epoch 5/5
212/212 [==============================] - 171s 806ms/step - loss: 0.0032 - accuracy: 1.0000


In [29]:
from sklearn.metrics import confusion_matrix , classification_report
y_pred = ann.predict(X_test)
y_pred_classes = [np.argmax(element) for element in y_pred]

print("Classification Report: \n", classification_report(y_test, y_pred_classes))

Classification Report: 
               precision    recall  f1-score   support

           0       1.00      1.00      1.00       115
           1       1.00      1.00      1.00       215
           2       1.00      1.00      1.00       134
           3       1.00      1.00      1.00       176
           4       1.00      1.00      1.00       300
           5       1.00      1.00      1.00       138
           6       1.00      1.00      1.00        95
           7       1.00      1.00      1.00       134
           8       1.00      1.00      1.00       197
           9       1.00      1.00      1.00       186

    accuracy                           1.00      1690
   macro avg       1.00      1.00      1.00      1690
weighted avg       1.00      1.00      1.00      1690



In [30]:
cnn = models.Sequential([
    layers.Conv2D(filters=32, kernel_size=(3, 3), activation='relu', input_shape=(160, 160, 3)),
    layers.MaxPooling2D((2, 2)),
    
    layers.Conv2D(filters=64, kernel_size=(3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(10, activation='softmax')
])

In [31]:
cnn.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [32]:
cnn.fit(X_train, y_train, epochs=10)

Epoch 1/10
212/212 [==============================] - 199s 935ms/step - loss: 0.1656 - accuracy: 0.9605
Epoch 2/10
212/212 [==============================] - 200s 943ms/step - loss: 4.1244e-05 - accuracy: 1.0000
Epoch 3/10
212/212 [==============================] - 199s 940ms/step - loss: 1.9109e-05 - accuracy: 1.0000
Epoch 4/10
212/212 [==============================] - 199s 940ms/step - loss: 1.1267e-05 - accuracy: 1.0000
Epoch 5/10
212/212 [==============================] - 205s 967ms/step - loss: 7.3835e-06 - accuracy: 1.0000
Epoch 6/10
212/212 [==============================] - 200s 944ms/step - loss: 5.0958e-06 - accuracy: 1.0000
Epoch 7/10
212/212 [==============================] - 201s 946ms/step - loss: 3.6206e-06 - accuracy: 1.0000
Epoch 8/10
212/212 [==============================] - 201s 947ms/step - loss: 2.7186e-06 - accuracy: 1.0000
Epoch 9/10
212/212 [==============================] - 201s 948ms/step - loss: 2.1563e-06 - accuracy: 1.0000
Epoch 10/10
212/212 [===========

In [33]:
y_pred = cnn.predict(X_test)
y_pred[:5]

array([[6.0344144e-19, 1.9608135e-20, 7.1562130e-07, 1.5076776e-13,
        9.9999928e-01, 1.8174973e-16, 1.6363377e-09, 2.4928709e-10,
        1.8987699e-11, 6.5182079e-21],
       [7.4307763e-08, 9.9716624e-12, 3.9716841e-12, 1.8158194e-11,
        4.5619162e-14, 9.9999440e-01, 2.9157770e-09, 1.0388696e-16,
        3.3140229e-10, 5.5069122e-06],
       [9.9999881e-01, 1.0373029e-07, 5.4754683e-08, 2.1010464e-09,
        4.5523446e-12, 2.8584624e-08, 1.5014106e-08, 5.0404791e-11,
        1.0029125e-06, 3.3900420e-09],
       [3.3825667e-09, 5.8560517e-14, 1.4573983e-09, 3.7419217e-11,
        4.0725982e-12, 1.0000000e+00, 6.3022414e-09, 3.0240126e-11,
        1.0584588e-08, 2.8687736e-11],
       [8.1216322e-13, 1.3915221e-12, 9.1676566e-10, 1.7571475e-12,
        3.8889648e-11, 1.1164779e-13, 1.9966578e-08, 4.4696965e-18,
        1.0000000e+00, 1.4901250e-13]], dtype=float32)

In [34]:
y_classes = [np.argmax(element) for element in y_pred]
y_classes[:5]

[4, 5, 0, 5, 8]

In [35]:
y_test[:5]

array([4, 5, 0, 5, 8])

In [36]:
classes[y_classes[3]]

'writing_on_board'

In [37]:
classes[y_classes[9]]

'Golf'

In [39]:
cnn.save('test_hra.hdf5')